In [ ]:
#-------------------------------------------------------------------------------
#
# Run this with: 
#     voila --no-browser --show_Tracebacks=True --VoilaConfiguration.file_whitelist="['.*.(nc|csv)']" iot_data_dashboard.ipynb
#
#     or
#
#     nohup voila --no-browser --show_Tracebacks=True --VoilaConfiguration.file_whitelist="['.*.(nc|csv|.png)']" iot_data_dashboard.ipynb >script.out 2>&1 &
#
# Without whitelisting the logos in the 'about' tab won't appear, and the data file download  
# will not work (error 403 'forbidden').
# Note that within Jupyter notebook/lab only text file download will work (will be opened and
# visualized in a new tab). But Jupyter doesn't know how to handle .nc files, and so gives you
# a pop-up error with a silly message ('File download error: the file is not utf-8 encoded').
# Download will work in voilà, provided that the files have been correctly whitelisted.
# In the next cell the calls to 'display' may be commented out when working in Jupyter: they are
# meant to avoid excessive whitespace on the page margins when running the dashboard in voilà.
#

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell {padding: 0 !important; }</style>"))
display(HTML("<style>.jp-Notebook {padding: 0 !important; }</style>"))
from netCDF4 import Dataset
import datetime
import dateutil
import base64
from fnmatch import fnmatch
import ipywidgets as widgets
import os
import numpy as np
from IPython.display import display, FileLink
import pymongo
import pandas as pd
import xarray as xr

In [ ]:
# constant definitions

MONGO_IP = 'localhost'
MONGO_PORT = 27017
DATABASE = 'stations'

DIRECTORY = '.'

In [ ]:
# connect to MongoDB
client = pymongo.MongoClient(host=MONGO_IP, port=MONGO_PORT)
db = client[DATABASE]

In [ ]:
class DataStore:
    """
    This class queries the database and holds a pandas dataframe which stores all of the queried data
    It is also capable of outputting the data as a .csv or .nc file.
    """
    def __init__(self, db: pymongo.database.Database, station_num: int = 1):

        # sets self.station to the appriopriate collection in the database
        self.station = db[f'station{station_num}']

        
        # sets self.conf to the config document in the station collection
        self.conf = db['stations_info'].find_one({'config': True, 'station_num': station_num})
        
        
        all_measurements = self.get_all_measurements()
        self.data = self.query(all_measurements)
        
        
        self.datetime = self.data['datetime']

        self.measurements = self.get_measurement_names()
        
        self.create_config()

    def get_all_measurements(self):
        """
        Queries the DB for graphable measurements and returns a list of strings in form sensor.measurement.index, i.e. particulate_matter.PM1count.1
        Excludes the gps data, and sensor, index, and type fields
        """
        measurements = []
        
        # chooses a non-config document
        doc = self.station.find_one({'config' : {"$exists" : False}})

        # iterates through each sensor in the document
        for sensor in doc:

            # excludes the object id field, datetime field, and gps sensor
            if sensor not in ['_id', 'datetime', 'gps']:
                sensor_fields = doc[sensor]

                # iterates through each field in the sensor, excluding the sensor name, index, and type/brand
                for field in sensor_fields:
                    if field not in ['sensor', 'index', 'type']:

                        # adds each field to a list in form [particulate_matter.PM1count.0, air_sensor.humidity.1]
                        measurement = f"{sensor_fields['sensor']}.{field}.{sensor_fields['index']}" 
                        measurements.append(measurement)

        return measurements 

    def query(self, measurements: list):
        """
        Runs an aggregation pipeline to query the database for the data given,
        then loads that data into a pandas dataframe
        
        measurements: a list of measurements in the form sensor.measurement.index. 
        can be generated by get_all_measurements()
        """
        
        # includes only files which have a datetime field
        exclude_config = {'$match': {
            'datetime': {'$exists': True}
            }
        }

        # sorts all documents by the datetime value, from earliest to latest
        sort_by_datetime = {'$sort': {
                'datetime': 1
            }
        }

        # unpacks the fields from the database format, i.e. 
        # {
        # datetime: val
        # particulate_matter+0: {
        #     PM1count: val,
        #     PM1mass: val,
        #     }
        # air_sensor+1: {
        #     humidity: val,
        #     temperature: val
        #     }
        # }
        # becomes:
        # {
        # datetime: val
        # PM1count+0: val
        # PM1mass+0: val
        # humidity+1: val
        # temperature+1: val
        # }
        unpack = {'$project': {
                '_id': 0, 
                'datetime': 1
            }
        }

        # adds each measurement in the measurements parameter to the unpack stage
        for measurement in measurements:
            sensor, field, index = measurement.split(".")
            unpack['$project'][f'{field}+{index}'] = f'${sensor}+{index}.{field}'

        #runs the aggregation
        aggr = self.station.aggregate([exclude_config, sort_by_datetime, unpack], allowDiskUse = True)
        
        #casts the aggregation as a list of dictionaries and then loads that list as a pandas dataframe
        df = pd.DataFrame(list(aggr))

        return df 
    
    def get_measurement_names(self):
        """
        Returns a set of measurements in the form set('PM1count','PM1mass',...,'temperature','humidity','co2')
        Used for the button labels
        """
        measurements = set()
        for key in self.data:
            if key != 'datetime':
                measurements.add(f"{key.split('+')[0]}")

        return measurements

    def get_series(self, key: str):
        """
        Returns the given series based on the name of its column. Identical to DataStore[column] or DataStore.data[column]
        """
        return self.data[key]
    
    def create_config(self):
        self.config = {}
        cols = self.data.columns
        
        for col in cols:
            if col in self.measurements:
                continue
            
            measure = col.split('+')[0]
            self.config[measure] = self.config.get(measure, 0) + 1

    def to_csv(self, filename: str, start_date: str = None, end_date: str = None, cols: list = None):   
        """
        Saves the dataframe as a .csv file
        
        filename: The name of the file.
        start_date (Optional): The first day of values to include. Defaults to the first date in the dataframe
        end_date (Optional): The last day of values to include. Defaults to the last date in the dataframe
        cols (Optional): A list of columns to include in the file. Defaults to all columns.
        
        to_csv(filename='january_data',start_date='2023-01',end_date='2023-01',cols=['tempeature+0',temperature+1'])
        will save january_data.csv, containing the temperature values from January 2023
        """
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            return self.data.set_index('datetime').loc[start_date:end_date,:].to_csv(f"{filename}", index=True, header=True)
        
        return self.data.set_index('datetime').loc[start_date:end_date,cols].to_csv(f"{filename}", index=True, header=True)
    
    def to_netcdf(self, filename: str, start_date: str = None, end_date: str = None, cols: list = None):
        """
        Saves the dataframe as a .nc file
        
        filename: The name of the file.
        start_date (Optional): The first day of values to include. Defaults to the first date in the dataframe
        end_date (Optional): The last day of values to include. Defaults to the last date in the dataframe
        cols (Optional): A list of columns to include in the file. Defaults to all columns.
        
        to_nc(filename='january_data', start_date='2023-01', end_date='2023-01', cols=['tempeature+0',temperature+1'])
        will save january_data.nc, containing the temperature values from January 2023
        """
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            x = xr.Dataset.from_dataframe(self.data.set_index('datetime').loc[start_date:end_date,:])
        else:
            x = xr.Dataset.from_dataframe(self.data.set_index('datetime').loc[start_date:end_date,cols])
            
        ### Per datum in the column, attributes need to be assigned, tentative list includes: full name, unit, sensor of origin, and various sensor specs
            
        return x.to_netcdf(f"{filename}")
    
    def to_json(self, start_date = None, end_date = None, cols = None):
        
        if start_date == None:
            start_date = self['datetime'].iloc[0]
        if end_date == None:
            end_date = self['datetime'].iloc[-1]
            
        if cols == None:
            return self.data.set_index('datetime').loc[start_date:end_date,:].to_json(date_format = "iso")

        return self.data.set_index('datetime').loc[start_date:end_date,cols].to_json(date_format = "iso")
        
    def __getitem__(self, key: str):
        return self.data[key]
    
    def average_series(self, measurement: str):
        """
        Given a measurement, returns a series with the average of all series' for that measurement.
        average_series('PM1count') will return the average of PM1count+0 and PM1count+1
        """
        return pd.concat([self[x] for x in self.data if measurement in x], axis=1).agg(np.mean, 1)
    

In [ ]:
class Plotter:
    '''
    Class to manage everything to do with plotting and plt
    '''
    
    DEFAULT_COLORS = {0: ['#8B0000', '#FF3131'],
                      1: ['#00008B', '#1F51FF'],
                      2: ['#008B00', '#39FF14'],
                      3: ['#8B8000', '#FFFF33']}
    
    def __init__(self, x_axis: iter, date_range_slider: widgets.SelectionRangeSlider) -> None:
        '''
        Initializes class
        @param x-axis Iterable containing x-axis elements. All plots managed by this plotter class must share
            the same x-axis
        @param date_range_slider Slider widget to control / limit the range of the x-axis
        '''
        
        # set up figure and plt settings
        plt.ioff()
        self.fig = plt.figure()
        self.fig.canvas.header_visible = False
        self.fig.canvas.resizable = False
        self.fig.canvas.toolbar_position = 'right'
        self.fig.canvas.layout.width = '100%'
        self.fig.set_figwidth(7)
        
        self.date_range_slider = date_range_slider
        
        # initialize variables
        self.x_axis = np.array(x_axis)
        self.axes = []
        self.colors = list(self.DEFAULT_COLORS.keys())  # keeps track of int for each default color
        self.max_graphs = len(self.colors)
        self.curr_graphs = 0


    def add_plot(self, data: iter, description: str) -> plt.Axes:
        '''
        Create a new subplot for the graph
        @param data Iterable of y-data to plot, len(data) must match len(self.x_axis)
        @param description What is being plotted, label for y-axis
        '''
        
        # check if we can add new plot
        # checks if data is compatible
        if (self.curr_graphs >= self.max_graphs) or \
            (data is None) or \
            (len(self.x_axis) != len(data)):
            return None
        
        if self.curr_graphs == 0:
            ax = self.fig.add_subplot()
        else:
            ax = self.axes[0].twinx()
            # pushes axis further away to not overlap
            ax.spines['right'].set_position(('outward', 
                                             50*(self.curr_graphs - 1)))
        
        # add graph description
        ax.description = description
        
        # plot
        ax.color = self.colors.pop()
        g_color = self.DEFAULT_COLORS[ax.color][0]  # new plots use the first color, subplots the second
        ax.plot(self.x_axis, data, '.',
               markersize=1, color=g_color)
        
        # edit axis info
        ax.set_ylabel(description, 
                        fontsize=12, color=g_color)
        ax.tick_params(axis='y', colors=g_color)
        self.fig.autofmt_xdate(rotation=45)
        
        
        self.curr_graphs += 1
        self.axes.append(ax)
        self.date_range_callback({'name': 'value'})
        return ax


    def add_subplot(self, data: iter, ax: plt.Axes) -> None:
        '''
        Adds new plot to an existing axis
        Only supports adding a single subplot per axes
        @param data Iterable of data to plot
        @param ax Existing plt.Axes object to graph
        '''
        g_color = self.DEFAULT_COLORS[ax.color][1]
        ax.plot(self.x_axis, data, '.', markersize=1, color=g_color)
        self.date_range_callback({'name': 'value'})


    def clear_plots(self) -> None:
        '''
        Clears all the plots and axes
        Resets the figure and list of available colors
        '''
        self.fig.clf()
        self.axes = []
        self.colors = list(self.DEFAULT_COLORS.keys())
        self.curr_graphs = 0
        #self.date_range_callback({'name': 'value'})
    
    
    def date_range_callback(self, wdic: dict) -> None:
        '''
        Callback for date range slider to edid min/max dates on graph
        '''
        
        if wdic['name'] != 'value':
            return
        #The right end of the date range needs to be rounded up to the next day
        min_day = self.date_range_slider.value[0]
        max_day = self.date_range_slider.value[1] + datetime.timedelta(days=1)
        try:
            self.axes[0].set_xlim((min_day, max_day))
        finally:
            self.finish_callback()
    
    
    def finish_callback(self):
        self.fig.tight_layout(pad=1.02)
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
    
        

In [ ]:
class ButtonList:
    '''
    This class groups all the buttons for different measurements of a sensor
    Manages the button callback functions and sending the appropriate info to the plotter object
    '''
    
    def __init__(self, data: DataStore, plotter: Plotter) -> None:
        '''
        Creates a new instance of a ButtonList
        @param data Instance of DataStore class. Contains the data to plot and also the attributes to 
            generate the buttons
        @param plotter Instance of Plotter class, manages the plotting of items
        '''
        
        # initiate class variables
        self.store = data
        self.plotter = plotter
        
        # collect info from DataStore object
        self.timeseries = self.store.datetime
        self.measurements = list(self.store.measurements)
        
        # init all buttons
        self.button_list = []
        self.active_buttons = []  # keeps track of currently active buttons
        for measurement in sorted(self.measurements):
            

            self.button_list.append(
                widgets.ToggleButton(
                    value = False,
                    description = measurement,
                    tooltip = measurement,
                    # tooltip=f"{self.store.data[ts][self.store.iLONG_NAME]} ({self.store.data[ts][self.store.iUNITS]})",
                    disabled=False,
                )
            )
            
            # add the callback function to the button
            self.button_list[-1].observe(self.callback)
            
            # button_list[-1]._Fidas_dashboard_units = self.store.data[ts][self.store.iUNITS]

        self.buttons = widgets.VBox(self.button_list)

        
    def callback(self, wdic: dict) -> None:
        '''
        Gets called when a button gets clicked
        Plots / clears the clicked button's measurments
        @param wdic Dictionary passed by the button. Contains at least the following keys:
            'type': type of notification
            If wdic['type'] is 'change' then the following keys are also passed
            'owner': the HasTraits instance
            'old': old value of the modified trait
            'new': new value of modified trait attribute
            'name': name of modified trait attribute
        '''
        
        # check if the trait changed is 'value'
        if wdic['name'] != 'value':
            return
        
        # check if data is being de-selected
        elif wdic['new'] == False:

            # remove element from the list of active buttons
            if wdic['owner'] in self.active_buttons:
                self.active_buttons.remove(wdic['owner'])
                
            else:
                return

            # clear plotter
            self.plotter.clear_plots()
            self.plot_graphs()  # plots all active buttons
            return
        
        # try and plot graph
        if not self.plot_graph(wdic['owner']):
            wdic['owner'].value = False  # change the value back to false if unable to plot it
        else:
            self.active_buttons.append(wdic['owner'])


    def plot_graphs(self) -> None:
        '''
        Plots all graphs in self.active_buttons
        '''
        for button in self.active_buttons:
            self.plot_graph(button)


    def plot_graph(self, button: widgets.ToggleButton) -> bool:
        '''
        Plots the data of the specified @param button
        If unable to plot it, returns False
        '''
        
        # get the attribute being plotted
        description = button.description
        
        # first collect the number of plots (1 or 2)
        num_plots = self.store.config.get(description, 0)
        
        # try plotting the first graph
        if num_plots == 0:
            # key isn't found in cofig
            ax = self.plotter.add_plot(self.store.data[description], description)
        else:
            # key is found in config
            ax = self.plotter.add_plot(self.store.average_series(description), description)
            #ax = self.plotter.add_plot(self.store.data[f'{description}+0'], description)
            
            
        if ax is None:
            return False
        
        # try plotting subplot if needed
#         if num_plots == 2:
#             self.plotter.add_subplot(self.store.data[f'{description}+1'], ax)
        
        return True
        

In [ ]:
#----------------------------------------------------------------------------------------------
#***Widgets for the intro/about tab***

In [ ]:
tabs = []

intro = widgets.HTML(
    value="""<p style="line-height: 150%">The Arabian Center for Climate and Environmental Sciences is currently
    hosting multiple environmental sensors at NYUAD, collecting particulate matter, temperature, pressure, and
    weather data. Using the tabs above, you can visualize data from these sensors. </p>
    <p>&nbsp;</p>""",
    layout=widgets.Layout(width='700px')
)
logo_ACCESS = widgets.HTML(
    value='<img src="ACCESS.png" alt="Arabian Center for Climate and Environmental Sciences" style="width:300px">',
    layout=widgets.Layout(
        margin='0 20px 0 20px'
    )
)

tab_about = widgets.VBox([intro, widgets.HBox([logo_ACCESS])])

tabs.append(tab_about)

In [ ]:
#Widgets for the station time series tabs

In [ ]:
for doc in db['stations_info'].find():
    if doc['station_num'] == 1 or doc['station_num'] == 2:
        data = DataStore(db,doc['station_num'])
        slider_days = np.unique([x.date() for x in data.datetime])
        date_range_slider = widgets.SelectionRangeSlider(
            options = slider_days,
            description = 'Date range:',
            orientation = 'horizontal',
            index = (0, len(slider_days)-1),
            disabled = False,
            continuous_update = False,
            tooltip = 'Select the date range to be plotted',
            layout=widgets.Layout(width='100%')
        )

        plotter = Plotter(data.datetime, date_range_slider)

        date_range_slider.observe(plotter.date_range_callback)

        button_list = ButtonList(data, plotter)
        
        decorated_canvas = widgets.VBox([date_range_slider,
                                 plotter.fig.canvas])
        tab_time_series = widgets.HBox([button_list.buttons, decorated_canvas])
        tabs.append(tab_time_series)

In [ ]:
#----------------------------------------------------------------------------------------------
#Widgets for the download tab

In [ ]:
DIRECTORY = '.'

files = os.listdir(DIRECTORY)
grouped_files = {}

for file in files:
    if file.endswith(".csv") or file.endswith(".nc"):
        station, file_type = file.split("_")[0].split(".")[0], file.split(".")[1]
        if station not in grouped_files:
            grouped_files[station] = {'csv':[],'nc':[]}
        grouped_files[station][file_type].append(file)

grouped_files = dict(sorted(grouped_files.items()))
vboxes = []

for station in grouped_files:
    csvs = [
    widgets.HTML(
        value='<u style="color:blue;">'+FileLink(f'{x}')._repr_html_()+'</u>',
        placeholder='',
        description='',
        tooltip='Click the link to download the file'
    ) for x in sorted(grouped_files[station]['csv'], key=lambda name: name[-11:-7]+name[-6:-4])
    ]
    ncs = [
    widgets.HTML(
        value='<u style="color:blue;">'+FileLink(f'{x}')._repr_html_()+'</u>',
        placeholder='',
        description='',
        tooltip='Click the link to download the file'
    ) for x in sorted(grouped_files[station]['nc'], key=lambda name: name[-10:-6]+name[-5:-3])
    ]
    hbox = widgets.HBox([widgets.VBox(csvs, layout=widgets.Layout(margin='0 20px 0 0')),
                         widgets.VBox(ncs, layout=widgets.Layout(margin='0 20px 0 0'))])
    label = widgets.HTML(
         value=f"<b>{station[0:7]} {station.replace('Station','')} Data</b><br>"
     )
    vbox = widgets.VBox([label, hbox],layout=widgets.Layout(margin='0 50px 0 0'))
    
    vboxes.append(vbox)

tab_downloads = widgets.HBox(vboxes)
tabs.append(tab_downloads)

In [ ]:
#----------------------------------------------------------------------------------------------
#***Display the tabbed interface***

In [ ]:
# create widget
tabbed_interface = widgets.Tab()

# set the tabs as tabs list
tabbed_interface.children = tabs

# set the title of the first tab as the About page
tabbed_interface.set_title(0, 'About')

# for doc in db['stations_info'].find():
#     station_num = doc['station_num']
#     if station_num != 0 and station_num != 2 and station_num != 3:
#         tabbed_interface.set_title(station_num, f'Sensor {station_num} time series')

#tabbed_interface.set_title(station_num-1, 'Data download')
display(tabbed_interface)